In [54]:
# Importamos las librerías

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin

# Configuramos el driver de chrome
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

titulos = []
precios = []
stocks = []
stars = []
images = []

url = "http://books.toscrape.com"

while True:
    titulos_sucios = []
    precios_sucios = []
    stocks_sucio = []
    stars_sucio = []
    images_sucio = []
    

    # urlaux = urljoin(url,f"/catalogue/page-{page}.html")

    driver.get(url)

    # Extracción de datos
    contenido = driver.page_source

    soup = bs(contenido)

    articles = soup.find_all("article",attrs={"class":"product_pod"})

    for libro in articles:
        titulo_paso1 = libro.find("h3").find("a")
        titulo = titulo_paso1["title"]
        titulos_sucios.append(titulo)
    
        precio = libro.find("p", attrs = {"class":"price_color"})
        precios_sucios.append(precio.text)
    
        stock = libro.find("p", attrs = {"class":"instock availability"})
        stocks_sucio.append(stock.text)
        
        star = libro.find("p", attrs = {"class":"star-rating"})
        rating = star["class"][1]
        stars_sucio.append(rating)
        
        imagen_tag = libro.find("img")
        imagen_relativa = imagen_tag["src"]
        imagen = urljoin(url, imagen_relativa)
        images_sucio.append(imagen)
    

    # Limpiando los datos
    for libro in titulos_sucios:
        titulos.append(libro.replace("\n","").strip())

    for libro in precios_sucios:
        precios.append(libro.replace("\n","").strip())

    for libro in stocks_sucio:
        stocks.append(libro.replace("\n","").strip())

    for libro in stars_sucio:
        stars.append(libro.replace("\n","").strip())

    for libro in images_sucio:
        images.append(libro.replace("\n","").strip())
    
    
   
    siguientepag = soup.select_one("li.next>a")
    if siguientepag:
        siguienteurl = siguientepag.get("href")
        url = urljoin(url, siguienteurl)
    else:
        break

# Mapeamos estrellas de texto a número
stars_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}
stars_numero = [stars_map.get(estrella, 0) for estrella in stars]


# Almacenamiento
df = pd.DataFrame({"Titulo":titulos,"Precio":precios,"Stock":stocks, "Stars":stars_numero, "Images":images})
df.head()

# Exportando los datos a un .csv
df.to_csv("libros_scraping.csv", index=False, encoding="utf-8")

1000
1000
1000
1000
